In [1]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

fatal: destination path 'IMDB-Movie-Reviews-Large-Dataset-50k' already exists and is not an empty directory.


In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [4]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype=str)

In [5]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype=str)

In [6]:
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [7]:
data_test

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
...,...,...
24995,This fanciful horror flick has Vincent Price p...,neg
24996,"The Intruder (L'Intrus), a film directed by Fr...",pos
24997,Holy crap. This was the worst film I have seen...,neg
24998,Clocking in at an interminable three hours and...,neg


In [8]:
data_test.shape, data_train.shape

((25000, 2), (25000, 2))

In [9]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_df=data_train, 
                                                                   text_column='Reviews',
                                                                   label_columns='Sentiment', 
                                                                   val_df=data_test,
                                                                   maxlen=500, #maximum limit for any reviews is 500 words
                                                                   preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [10]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [11]:
learner = ktrain.get_learner(model=model,
                   train_data=(x_train,y_train),
                   val_data=(x_test,y_test),
                   batch_size=6)

In [12]:
#find out the best learning rate
#learner.lr_find()
#learner.lr_plot()

In [13]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 4762s 1s/step - loss: 0.2474 - accuracy: 0.8971 - val_loss: 0.0978 - val_accuracy: 0.9690


In [15]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [16]:
data = ["The Shape of Water is an outstanding film that has something for everyone. The visuals are stunning, the plot is captivating, and the cast is excellent. I was thoroughly impressed by the film and would highly recommend it to anyone looking for a unique and entertaining movie experience.",
        "Wonder Woman is an inspiring and entertaining film that is sure to please audiences of all ages. The action sequences are thrilling, the story is engaging, and the performances are top-notch. It's a must-see for fans of superhero movies, as it delivers a powerful message and leaves viewers feeling uplifted and inspired.",
        "I was really disappointed with The Mummy. The plot was convoluted and confusing, the special effects were subpar, and the acting was wooden. I would not recommend this movie to anyone, as it was a huge waste of time.",
        "The Last Airbender was a terrible movie. The plot was overly long and confusing, the dialogue was cringe-worthy, and the acting was abysmal. All in all, it was two hours of my life I'll never get back. Avoid this one at all costs."]

In [17]:
predictor.predict(data)

1/1 [==============================] - 9s 9s/step


['pos', 'pos', 'neg', 'neg']

In [18]:
predictor.predict(data, return_proba=True)

1/1 [==============================] - 0s 282ms/step


array([[0.00216247, 0.9978376 ],
       [0.00231953, 0.9976804 ],
       [0.9982449 , 0.00175513],
       [0.99872535, 0.00127461]], dtype=float32)

In [19]:
predictor.get_classes()

['neg', 'pos']

In [20]:
#save and load model
predictor.save('/content/bert')

In [21]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.preproc (deflated 48%)
  adding: content/bert/tf_model.h5 (deflated 11%)


In [22]:
predictor_load = ktrain.load_predictor('/content/bert')

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [23]:
predictor_load.get_classes()

['neg', 'pos']

In [24]:
data_test_load = ["The Dark Knight is a masterpiece of modern cinema. The visuals are stunning, the action scenes are thrilling, and the performances are remarkable. It's a must-see for fans of superhero movies and a great choice for anyone looking for an entertaining and thought-provoking film experience.",
                  "I was incredibly disappointed with Alien: Covenant. The plot was unfocused and confusing, the special effects were unimpressive, and the dialogue was clunky. I would not recommend this movie to anyone, as it was an utter waste of time.",
                  " Lady Bird is an excellent coming-of-age story that is sure to tug at the heartstrings. The story is captivating, the characters are likeable, and the performances are outstanding. It's a must-watch for anyone looking for a poignant, emotional ride.",
                  "Fifty Shades of Grey was a terrible movie. The story was dull and dragged on forever, the dialogue was cringe-inducing, and the acting was wooden. I would advise anyone thinking of seeing this movie to save their money and stay away."]

In [25]:
predictor_load.predict(data_test_load)

1/1 [==============================] - 7s 7s/step


['pos', 'neg', 'pos', 'neg']